In [1]:
# data analysis stack
import pandas as pd
import numpy as np

# data visualization stack
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # set seaborn as default style

# data pre-processing stack
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    PolynomialFeatures
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#machine learning stack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import GridSearchCV

# miscellaneous
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
bike = pd.read_csv("./data/bike_train.csv", index_col=0, parse_dates=True)
bike.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


#### Log transformation is done because data is not normally distributed. 

In [3]:
bike["log_casual"]=np.log1p(bike["casual"])
bike["log_registered"]=np.log1p(bike["registered"])

#### Datetime column is splited and value 4 in the weather column is replaced with 3 since there is only one value.

In [4]:
def yoda(df):
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["hour"] = df.index.hour
    df["weather"].replace([4], [3], inplace = True)
    return df

In [5]:
yoda(bike)
bike.head(3)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,log_casual,log_registered,year,month,hour
datetime,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1.386294,2.639057,2011,1,0
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2.197225,3.496508,2011,1,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1.791759,3.332205,2011,1,2


#### To see the trend in the data we will create a column which index the month from 1 to 24.

In [6]:
def month_index(year, month):
    return (year-2011)*12 + month
    
bike['month_idx'] = month_index(bike["year"], bike["month"])

In [7]:
bike['day_of_week'] = bike.index.weekday + 1
bike.head(3)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,log_casual,log_registered,year,month,hour,month_idx,day_of_week
datetime,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1.386294,2.639057,2011,1,0,1,6
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2.197225,3.496508,2011,1,1,1,6
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1.791759,3.332205,2011,1,2,1,6


### CASUAL USERS
#### Casual Train

In [8]:
train,test = train_test_split(bike, test_size=0.2, random_state=101)

In [9]:
# we need to delete either the "temp" or the "atemp" because they are highly corelated with each other.
numerical_features = [
     'temp',
     #'atemp', ## temp was better than atemp. Training score is 2 points more 
     'humidity', ## add alot like 3 points
     'windspeed', # added some more power
     'month_idx',
     'hour' 
]

categorical_features = [
    #'season', Adding season does not add much. Mostly weather matters.
    'holiday',
    'workingday',
    'day_of_week',
    'weather' # our as a categorical training score is 78.
]

features = numerical_features + categorical_features

target_variable = 'log_casual' # for registered it it aroun 20 but casual gets 45 with these features except atemp.


In [10]:
X_casual_train,y_casual_train = train[features], train[target_variable]

In [11]:
# scaling and polynomial features
numerical_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('polynomial', PolynomialFeatures())
    ]
)

In [12]:
# one-hot encoding
categorical_transformer = Pipeline(
    steps=[
        ('ohe', OneHotEncoder(drop='first'))
    ]
)

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [14]:
estimator = Pipeline(
    steps=[
        ('preprocessor', preprocessor),   # preprocessing step
        ('lasso', Lasso()) # lasso regression
    ]
)

In [15]:
# Various hyperparameters and their possible values.
param_grid = {
    'preprocessor__num__polynomial__degree': [4],
    'preprocessor__num__polynomial__interaction_only': [False, True],
    'lasso__alpha': [100.,10.,1.,0.1,0.01],
    'lasso__max_iter': [5_000, 10_000,20_000]
}

In [16]:
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=5, 
    n_jobs=-1,
    verbose=1
)

In [17]:
# initial time
ti = time.time()

# grid-search cross-validation
gscv.fit(X_casual_train,y_casual_train)

# final time 
tf = time.time()

# time taken
print(f"time taken: {round(tf-ti,2)} sec")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
time taken: 9.74 sec


In [18]:
gscv.cv_results_

{'mean_fit_time': array([0.20506382, 0.07411079, 0.12701688, 0.07458119, 0.11514764,
        0.05566506, 0.10189123, 0.0624599 , 0.12783875, 0.07136965,
        0.11498594, 0.06375799, 0.09562712, 0.05238986, 0.09438586,
        0.05238295, 0.09665666, 0.05073204, 0.15000539, 0.05828428,
        0.14675121, 0.05903959, 0.14888825, 0.05667815, 0.42538037,
        0.05969181, 0.46767025, 0.08115001, 0.53176112, 0.07655768]),
 'std_fit_time': array([0.06858907, 0.04061184, 0.01488966, 0.00372964, 0.00839424,
        0.00308083, 0.00484684, 0.00498453, 0.00690241, 0.00369746,
        0.00473385, 0.00318333, 0.0038421 , 0.00219258, 0.00421632,
        0.00310533, 0.00615467, 0.00199898, 0.0122234 , 0.00502423,
        0.00781747, 0.00358742, 0.00962257, 0.00446927, 0.03112486,
        0.00412913, 0.02559567, 0.00594353, 0.07650941, 0.01434085]),
 'mean_score_time': array([0.03560452, 0.0309617 , 0.02973533, 0.02021084, 0.02735977,
        0.01692896, 0.02563219, 0.02119737, 0.03159685, 0.02

In [19]:
# list of columns to show
column_list = ['param_preprocessor__num__polynomial__degree',
               'param_preprocessor__num__polynomial__interaction_only',
               'param_lasso__alpha',
               'param_lasso__max_iter',
               'mean_test_score',
               'std_test_score',
               'rank_test_score'
              ]
# create result dataframe
result_df = pd.DataFrame(gscv.cv_results_)[column_list]

# rename columns
result_df.rename(
    columns=lambda name: name.split('__')[-1],inplace=True
)

# order by rank
result_df.sort_values(
    by='rank_test_score', ascending=True, inplace=True, ignore_index=True
)

result_df

,degree,interaction_only,alpha,max_iter,mean_test_score,std_test_score,rank_test_score
0,4,False,0.01,10000,0.817814,0.007092,1
1,4,False,0.01,5000,0.817814,0.007092,1
2,4,False,0.01,20000,0.817814,0.007092,1
3,4,False,0.1,20000,0.657246,0.006787,4
4,4,False,0.1,10000,0.657246,0.006787,4
5,4,False,0.1,5000,0.657246,0.006787,4
6,4,True,0.01,20000,0.618719,0.010540,7
7,4,True,0.01,10000,0.618719,0.010540,7
8,4,True,0.01,5000,0.618719,0.010540,7
9,4,True,0.1,20000,0.561331,0.007897,10


In [20]:
gscv.best_params_

{'lasso__alpha': 0.01,
 'lasso__max_iter': 5000,
 'preprocessor__num__polynomial__degree': 4,
 'preprocessor__num__polynomial__interaction_only': False}

In [21]:
round(gscv.best_score_,6)

0.817814

In [22]:
best_model_casual = gscv.best_estimator_
best_model_casual

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=4))]),
                                                  ['temp', 'humidity',
                                                   'windspeed', 'month_idx',
                                                   'hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['holiday', 'workingday',
                                                   'day_of_week',
                                                   'weather'])])),
                ('lasso', Lasso(alpha=0.01, max_iter=5000))])

In [23]:
best_model_casual.fit(X_casual_train,y_casual_train);

In [24]:
# training score
casual_training_score = best_model_casual.score(X_casual_train, y_casual_train)

print(f'Casual Train score: {round(casual_training_score,6)}')


Casual Train score: 0.821153


#### Casual Test 

In [25]:
X_casual_test,y_casual_test = test[features], test[target_variable]

In [26]:
best_model_casual.fit(X_casual_test,y_casual_test);

In [27]:
# training score
casual_test_score = best_model_casual.score(X_casual_test, y_casual_test)

print(f'Casual Test score: {round(casual_test_score,6)}')

Casual Test score: 0.822295


**Casual total prediction**

In [28]:
X_casual_total, y_casual_total = bike[features], bike[target_variable]

In [29]:
y_casual_pred = best_model_casual.predict(X_casual_total)
y_casual_pred

array([1.48057208, 0.9511494 , 0.73362157, ..., 2.27855827, 2.25470619,
       2.07337434])

### REGISTERED USERS
#### Registered train

In [30]:
# we need to delete either the "temp" or the "atemp" because they are highly corelated with each other.
numerical_features = [
     'temp',
     #'atemp', ## temp was better than atemp. Training score is 2 points more 
     'humidity', ## add alot like 3 points
     'windspeed', # added some more power
     'month_idx',
     'hour' 
]

categorical_features = [
    #'season', Adding season does not add much. Mostly weather matters.
    'holiday',
    'workingday',
    'day_of_week',
    'weather' # our as a categorical training score is 78.
]

features = numerical_features + categorical_features

target_variable = 'log_registered' # for registered it it aroun 20 but casual gets 45 with these features except atemp.


In [31]:
X_registered_train, y_registered_train = train[features], train[target_variable]

In [32]:
# scaling and polynomial features
numerical_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('polynomial', PolynomialFeatures())
    ]
)

In [33]:
# one-hot encoding
categorical_transformer = Pipeline(
    steps=[
        ('ohe', OneHotEncoder(drop='first'))
    ]
)

In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [35]:
estimator = Pipeline(
    steps=[
        ('preprocessor', preprocessor),   # preprocessing step
        ('lasso', Lasso()) # lasso regression
    ]
)

In [36]:
param_grid = {
    'preprocessor__num__polynomial__degree': [5],
    'preprocessor__num__polynomial__interaction_only': [False,True],
    'lasso__alpha': [100.,10.,1.,0.1,0.01],
    'lasso__max_iter': [5_000, 10_000,20_000]
}

In [37]:
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=5, 
    n_jobs=-1,
    verbose=1
)

In [38]:
# initial time
ti = time.time()

# grid-search cross-validation
gscv.fit(X_registered_train,y_registered_train)

# final time 
tf = time.time()

# time taken
print(f"time taken: {round(tf-ti,2)} sec")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
time taken: 23.24 sec


In [39]:
gscv.cv_results_

{'mean_fit_time': array([0.21444826, 0.05525088, 0.17759361, 0.05154824, 0.16311617,
        0.05736146, 0.15301776, 0.05199275, 0.15514984, 0.05343046,
        0.15034204, 0.05123677, 0.18141074, 0.05356193, 0.18048024,
        0.05390167, 0.18126216, 0.05248098, 0.46794672, 0.05913177,
        0.47126775, 0.05789061, 0.46995311, 0.06383586, 3.84466124,
        0.06848755, 4.11582799, 0.07848673, 3.56703653, 0.07819657]),
 'std_fit_time': array([0.04233224, 0.00685311, 0.02059066, 0.00265678, 0.0076232 ,
        0.00956674, 0.01183659, 0.00258953, 0.01294023, 0.00369797,
        0.01003014, 0.00379757, 0.01007854, 0.00155327, 0.00933913,
        0.00287859, 0.01224505, 0.00454686, 0.06018562, 0.00527763,
        0.0555141 , 0.00224616, 0.04839717, 0.00391268, 0.06633414,
        0.01470491, 0.17931126, 0.00989762, 0.69164114, 0.0065156 ]),
 'mean_score_time': array([0.05219588, 0.01545296, 0.03924818, 0.01531711, 0.03253384,
        0.01705923, 0.02757168, 0.01498265, 0.02952018, 0.01

In [40]:
# list of columns to show
column_list = ['param_preprocessor__num__polynomial__degree',
               'param_preprocessor__num__polynomial__interaction_only',
               'param_lasso__alpha',
               'param_lasso__max_iter',
               'mean_test_score',
               'std_test_score',
               'rank_test_score'
              ]
# create result dataframe
result_df = pd.DataFrame(gscv.cv_results_)[column_list]

# rename columns
result_df.rename(
    columns=lambda name: name.split('__')[-1],inplace=True
)

# order by rank
result_df.sort_values(
    by='rank_test_score', ascending=True, inplace=True, ignore_index=True
)

result_df

,degree,interaction_only,alpha,max_iter,mean_test_score,std_test_score,rank_test_score
0,5,False,0.01,10000,0.741149,0.012144,1
1,5,False,0.01,5000,0.741149,0.012144,1
2,5,False,0.01,20000,0.741149,0.012144,1
3,5,False,0.1,20000,0.610024,0.014701,4
4,5,False,0.1,10000,0.610024,0.014701,4
5,5,False,0.1,5000,0.610024,0.014701,4
6,5,True,0.01,20000,0.465801,0.019683,7
7,5,True,0.01,10000,0.465801,0.019683,7
8,5,True,0.01,5000,0.465801,0.019683,7
9,5,True,0.1,20000,0.441790,0.017672,10


In [41]:
gscv.best_params_

{'lasso__alpha': 0.01,
 'lasso__max_iter': 5000,
 'preprocessor__num__polynomial__degree': 5,
 'preprocessor__num__polynomial__interaction_only': False}

In [42]:
round(gscv.best_score_,6)

0.741149

In [43]:
best_model_registered = gscv.best_estimator_
best_model_registered

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=5))]),
                                                  ['temp', 'humidity',
                                                   'windspeed', 'month_idx',
                                                   'hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['holiday', 'workingday',
                                                   'day_of_week',
                                                   'weather'])])),
                ('lasso', Lasso(alpha=0.01, max_iter=5000))])

In [44]:
best_model_registered.fit(X_registered_train,y_registered_train);

In [45]:
# training score
registered_train_score = best_model_registered.score(X_registered_train, y_registered_train)

print(f'Registered Train score: {round(registered_train_score,6)}')


Registered Train score: 0.746643


#### Registered Test

In [46]:
X_registered_test,y_registered_test = test[features], test[target_variable]

In [47]:
best_model_registered.fit(X_registered_test,y_registered_test);

In [48]:
# test score
registered_test_score = best_model_registered.score(X_registered_test, y_registered_test)

print(f'Registered Test score: {round(registered_test_score,6)}')


Registered Test score: 0.765034


#### Registered total prediction
Given the forecasted weather conditions, how many bicycles can we expect to be rented out (city-wide) this Saturday at 2pm?

In [49]:
X_registered_total, y_registered_total = bike[features], bike[target_variable]

In [50]:
y_registered_pred = best_model_registered.predict(X_registered_total)
y_registered_pred

array([2.20920215, 1.1780434 , 0.92263077, ..., 5.49387229, 5.14107308,
       4.23005365])

In [51]:
magic_day = pd.DataFrame({
    "datetime": [pd.to_datetime("20230415-14:00:00")],
    "season": [1],
    "holiday": [0],
    "workingday": [0],
    "weather": [3],
    "temp": [23.3],
    "atemp": [24],
    "humidity": [76],
    "windspeed": [5],
    "casual": [""],
    "registered": [""],
    "count": [""],
}).set_index("datetime")

In [52]:
magic_day = yoda(magic_day)
magic_day

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour
datetime,,,,,,,,,,,,,,
2023-04-15 14:00:00,1,0,0,3,23.3,24,76,5,,,,2023,4,14


In [53]:
magic_day['month_idx'] =  magic_day["month"]

In [54]:
magic_day['day_of_week'] = magic_day.index.weekday + 1
magic_day.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,month_idx,day_of_week
datetime,,,,,,,,,,,,,,,,
2023-04-15 14:00:00,1,0,0,3,23.3,24,76,5,,,,2023,4,14,4,6


In [55]:
X_magic_casual = magic_day[features]
X_magic_register = magic_day[features]

In [56]:
y_pred_casual = best_model_casual.predict(X_magic_casual)
y_pred_casual

array([4.26775084])

In [57]:
y_pred_register = best_model_registered.predict(X_magic_casual)
y_pred_register

array([4.75945651])

In [58]:
y_pred_casual_nonlog = np.exp(y_pred_casual)-1
y_pred_casual_nonlog

array([70.36095266])

In [59]:
y_pred_register_nonlog = np.exp(y_pred_register)-1
y_pred_register_nonlog

array([115.68249315])

In [60]:
magic_day_count = y_pred_casual_nonlog + y_pred_register_nonlog
magic_day_count 

array([186.04344581])